In [1]:
'''
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Do this only in Colab notebooks! Otherwise use pip install unsloth
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf "datasets>=3.4.1,<4.0.0" "huggingface_hub>=0.34.0" hf_transfer
    !pip install --no-deps unsloth

# Install latest transformers for Gemma 3N
!pip install --no-deps --upgrade timm # Only for Gemma 3N
'''

'\nimport os\nif "COLAB_" not in "".join(os.environ.keys()):\n    !pip install unsloth\nelse:\n    # Do this only in Colab notebooks! Otherwise use pip install unsloth\n    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl triton cut_cross_entropy unsloth_zoo\n    !pip install sentencepiece protobuf "datasets>=3.4.1,<4.0.0" "huggingface_hub>=0.34.0" hf_transfer\n    !pip install --no-deps unsloth\n\n# Install latest transformers for Gemma 3N\n!pip install --no-deps --upgrade timm # Only for Gemma 3N\n'

In [2]:
from unsloth import FastModel
import torch
from transformers import AutoProcessor, AutoModelForCausalLM
from PIL import Image
import requests
from io import BytesIO
torch._dynamo.config.cache_size_limit = 512  # Default is 64, increase as needed

model, tokenizer = FastModel.from_pretrained(
  #  model_name = "unsloth/gemma-3n-E2B-it-litert-preview",
  # model_name = "mekpro/gemma-3n-botanist-observe6-merged",
    model_name = "mekpro/gemma3-botanist-merged",
    dtype = torch.bfloat16, # None for auto detection
    max_seq_length = 512, # Choose any for long context!
    load_in_4bit = False,  # 4 bit quantization to reduce memory
    full_finetuning = False, # [NEW!] We have full finetuning now!
)



🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.8.1: Fast Gemma3 patching. Transformers: 4.54.0.
   \\   /|    NVIDIA L4. Num GPUs = 1. Max memory: 22.161 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.9. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: QLoRA and full finetuning all not selected. Switching to 16bit LoRA.


model.safetensors.index.json: 0.00B [00:00, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.96G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.64G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

...

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/4.69M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/33.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/35.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/670 [00:00<?, ?B/s]

In [3]:
def do_gemma_3n_inference(messages, max_new_tokens = 256):
    inputs = tokenizer.apply_chat_template(
        messages,
        add_generation_prompt = True,  # Must add for generation
        tokenize = True,
        return_dict = True,
        return_tensors = "pt",
    ).to("cuda")

    output_ids = model.generate(
        **inputs,
        max_new_tokens = max_new_tokens,
        temperature = 0.8,
        top_p = 0.95,
        top_k = 64,
    )
    generated_ids = output_ids[0][inputs['input_ids'].shape[1]:]
    generated_text = tokenizer.decode(generated_ids, skip_special_tokens=True)

    return generated_text

#link = "https://raw.githubusercontent.com/plantnet/PlantNet-300K/refs/heads/main/images/1.jpg" # Cirsium arvense
link = "https://mekpro.ap-south-1.linodeobjects.com/pelargonium_peltatum_4.jpg"
instruction = "As a botanist, observe the image of the flower and describe its visual features and species_name in JSON format. {color, inflorescencetype, inflorescence_description, flower_arrangement, flower_density, species, family, genus}"
FastModel.for_inference(model)

messages = [{
    "role" : "user",
    "content": [
        { "type": "image", "image" : link },
        { "type": "text",  "text" : instruction }
    ]
}]

# You might have to wait 1 minute for Unsloth's auto compiler
#print(do_gemma_3n_inference(messages, max_new_tokens = 256))

In [4]:
import csv
import json
import urllib.request
from io import StringIO

def llm_inference_tester(testsize=None):  # Added testsize parameter
    # Configuration
    csv_url = "https://mekpro.ap-south-1.linodeobjects.com/botanist/random_sample_100.csv"
    image_domain_prefix = "https://mekpro.ap-south-1.linodeobjects.com/botanist/"

    # Download and read CSV
    print("Downloading CSV file...")
    with urllib.request.urlopen(csv_url) as response:
        csv_content = response.read().decode('utf-8')

    # Parse CSV
    csv_reader = csv.DictReader(StringIO(csv_content))

    # Initialize counters
    total_count = 0
    correct_species = 0
    correct_inflorescence = 0
    invalid_json_count = 0  # Track invalid JSON responses

    # Prepare instruction template
    instruction = ("As a botanist, observe the image of the flower and short describe (total <100 words) its visual features "
                  "and species_name in JSON format. {'color':'', 'inflorescencetype':'', 'inflorescence_description':'', "
                  "'flower_arrangement':'', 'flower_density':'', 'species':'', 'family':'', 'genus':''}")


    # Process each row
    print(f"\nProcessing images{f' (limit: {testsize})' if testsize else ''}...")

    for idx, row in enumerate(csv_reader):
        # Check if we've reached the test size limit
        if testsize and idx >= testsize:
            break

        total_count += 1

        # Construct full image URL
        image_url = image_domain_prefix + row['filepath']

        # Prepare messages for inference
        messages = [{
            "role": "user",
            "content": [
                {"type": "image", "image": image_url},
                {"type": "text", "text": instruction}
            ]
        }]

        # Get inference result
        try:
            result_string = do_gemma_3n_inference(messages, max_new_tokens=256)
            # Parse JSON result
            try:
                result_string = result_string.replace("```json", '').replace("```",'').replace("\n","")
                print(result_string)
                result_json = json.loads(result_string)

                # Extract predicted values
                predicted_species = result_json.get('species', '').strip()
                predicted_inflorescence = result_json.get('inflorescencetype', '').strip()

                # Ground truth values
                true_species = row['species'].strip()
                true_inflorescence = row['inflorescencetype'].strip()

                # Check species match with partial credit for genus
                if predicted_species.lower() == true_species.lower():
                    species_match = 1
                elif predicted_species and true_species:
                    # Check if genus (first part) matches
                    pred_genus = predicted_species.split()[0].lower() if predicted_species.split() else ""
                    true_genus = true_species.split()[0].lower() if true_species.split() else ""
                    species_match = 0.5 if pred_genus and pred_genus == true_genus else 0
                else:
                    species_match = 0

                correct_species += species_match

                # Check inflorescence type match
                inflorescence_match = 1 if predicted_inflorescence.lower() == true_inflorescence.lower() else 0
                correct_inflorescence += inflorescence_match

                # Print result for this image
                print(f"Image {total_count}: {row['filepath']}")
                print(f"  True species: {true_species} | Predicted: {predicted_species} | Score: {species_match}")
                print(f"  True inflorescence: {true_inflorescence} | Predicted: {predicted_inflorescence} | Score: {inflorescence_match}")

            except json.JSONDecodeError:
                # Broken JSON - give 0 score
                invalid_json_count += 1  # Count invalid JSON
                print(f"Image {total_count}: {row['filepath']}")
                print(f"  Error: Invalid JSON response")
                print(f"  Species score: 0 | Inflorescence score: 0")

        except Exception as e:
            # Handle any other errors
            print(f"Image {total_count}: {row['filepath']}")
            print(f"  Error during inference: {str(e)}")
            print(f"  Species score: 0 | Inflorescence score: 0")

        print("-" * 80)

    # Calculate and display summary
    print("\n" + "=" * 80)
    print("SUMMARY RESULTS")
    print("=" * 80)
    print(f"Total images tested: {total_count}")
    print(f"Invalid JSON responses: {invalid_json_count}")  # Show invalid JSON count
    print(f"\nSpecies accuracy:")
    print(f"  Correct: {correct_species}/{total_count}")
    print(f"  Accuracy: {correct_species/total_count*100:.2f}%")
    print(f"\nInflorescence type accuracy:")
    print(f"  Correct: {correct_inflorescence}/{total_count}")
    print(f"  Accuracy: {correct_inflorescence/total_count*100:.2f}%")
    print(f"\nOverall performance:")
    print(f"  Total correct predictions: {correct_species + correct_inflorescence}/{total_count * 2}")
    print(f"  Average accuracy: {(correct_species + correct_inflorescence)/(total_count * 2)*100:.2f}%")


#llm_inference_tester(testsize=50)

In [5]:
import csv
import json
import urllib.request
from io import StringIO

def llm_inference_tester(testsize=None):  # Added testsize parameter
    # Configuration
    csv_url = "https://mekpro.ap-south-1.linodeobjects.com/botanist/random_sample_100.csv"
    image_domain_prefix = "https://mekpro.ap-south-1.linodeobjects.com/botanist/"

    # Download and read CSV
    print("Downloading CSV file...")
    with urllib.request.urlopen(csv_url) as response:
        csv_content = response.read().decode('utf-8')

    # Parse CSV
    csv_reader = csv.DictReader(StringIO(csv_content))

    # Initialize counters
    total_count = 0

    # Top-1 counters (first prediction only)
    correct_species_top1 = 0        # Full species match in first prediction
    half_correct_species_top1 = 0   # Only genus match in first prediction
    correct_inflorescence_top1 = 0  # Inflorescence match in first prediction

    # Top-5 species counters
    correct_species_top5 = 0        # Full species match in top-5
    half_correct_species_top5 = 0   # Only genus match in top-5
    # Top-5 inflorescence counters
    correct_inflorescence_top5 = 0
    invalid_json_count = 0  # Track invalid JSON responses

    # Prepare instruction template
    instruction = ("As a botanist, observe the image of the flower and short describe (total <100 words) its visual features "
                  "and species_name in JSON format. {'color':'', 'inflorescencetype':'', 'inflorescence_description':'', "
                  "'flower_arrangement':'', 'flower_density':'', 'species':'', 'family':'', 'genus':''}")

    # Process each row
    print(f"\nProcessing images{f' (limit: {testsize})' if testsize else ''}...")

    for idx, row in enumerate(csv_reader):
        # Check if we've reached the test size limit
        if testsize and idx >= testsize:
            break

        total_count += 1

        # Construct full image URL
        image_url = image_domain_prefix + row['filepath']

        # Prepare messages for inference
        messages = [{
            "role": "user",
            "content": [
                {"type": "image", "image": image_url},
                {"type": "text", "text": instruction}
            ]
        }]

        # Ground truth values
        true_species = row['species'].strip()
        true_inflorescence = row['inflorescencetype'].strip()
        true_genus = true_species.split()[0].lower() if true_species.split() else ""

        # Collect 5 predictions
        predictions = []
        predicted_species_list = []
        predicted_inflorescence_list = []

        print(f"\nImage {total_count}: {row['filepath']}")
        print(f"True species: {true_species} | True inflorescence: {true_inflorescence}")
        print("Getting 5 predictions...")

        for pred_num in range(5):
            try:
                # Get inference result
                result_string = do_gemma_3n_inference(messages, max_new_tokens=256)

                # Parse JSON result
                try:
                    result_string = result_string.replace("```json", '').replace("```",'').replace("\n","")
                    result_json = json.loads(result_string)

                    # Extract predicted values
                    predicted_species = result_json.get('species', '').strip()
                    predicted_inflorescence = result_json.get('inflorescencetype', '').strip()

                    # Store predictions
                    predicted_species_list.append(predicted_species)
                    predicted_inflorescence_list.append(predicted_inflorescence)

                    print(f"  Prediction {pred_num + 1}: Species: {predicted_species} | Inflorescence: {predicted_inflorescence}")

                except json.JSONDecodeError:
                    # Invalid JSON - store empty predictions
                    invalid_json_count += 1
                    predicted_species_list.append("")
                    predicted_inflorescence_list.append("")
                    print(f"  Prediction {pred_num + 1}: Invalid JSON response")

            except Exception as e:
                # Handle any other errors - store empty predictions
                predicted_species_list.append("")
                predicted_inflorescence_list.append("")
                print(f"  Prediction {pred_num + 1}: Error during inference: {str(e)}")

        # Check TOP-1 accuracy (first prediction only)
        if predicted_species_list and predicted_species_list[0]:
            first_pred_species = predicted_species_list[0]
            # Check full species match
            if first_pred_species.lower() == true_species.lower():
                correct_species_top1 += 1
            # Check genus match if no full match
            elif true_genus:
                first_pred_genus = first_pred_species.split()[0].lower() if first_pred_species.split() else ""
                if first_pred_genus == true_genus:
                    half_correct_species_top1 += 1

        # Check TOP-1 inflorescence
        if predicted_inflorescence_list and predicted_inflorescence_list[0]:
            if predicted_inflorescence_list[0].lower() == true_inflorescence.lower():
                correct_inflorescence_top1 += 1

        # Check if true species is in top-5 predictions
        species_full_match = False
        species_genus_match = False

        for pred_species in predicted_species_list:
            if pred_species and pred_species.lower() == true_species.lower():
                species_full_match = True
                break

        # If no full match, check for genus match
        if not species_full_match and true_genus:
            for pred_species in predicted_species_list:
                if pred_species:
                    pred_genus = pred_species.split()[0].lower() if pred_species.split() else ""
                    if pred_genus == true_genus:
                        species_genus_match = True
                        break

        # Update species counters
        if species_full_match:
            correct_species_top5 += 1
        elif species_genus_match:
            half_correct_species_top5 += 1

        # Check if true inflorescence is in top-5 predictions
        inflorescence_match = False
        for pred_inflorescence in predicted_inflorescence_list:
            if pred_inflorescence and pred_inflorescence.lower() == true_inflorescence.lower():
                inflorescence_match = True
                break

        if inflorescence_match:
            correct_inflorescence_top5 += 1

        # Print results for this image
        print(f"\nResults:")
        print(f"  Species: {'Full match in top-5' if species_full_match else 'Genus match only in top-5' if species_genus_match else 'No match in top-5'}")
        print(f"  Inflorescence: {'Match in top-5' if inflorescence_match else 'No match in top-5'}")
        print("-" * 80)

    # Calculate and display summary
    print("\n" + "=" * 80)
    print("SUMMARY RESULTS")
    print("=" * 80)
    print(f"Total images tested: {total_count}")
    print(f"Invalid JSON responses: {invalid_json_count} (out of {total_count * 5} total predictions)")

    # TOP-1 RESULTS
    print("\n" + "-" * 40)
    print("TOP-1 ACCURACY (First prediction only)")
    print("-" * 40)

    print(f"\nSpecies Top-1 accuracy:")
    print(f"  Full correct: {correct_species_top1}/{total_count}")
    print(f"  Full accuracy: {correct_species_top1/total_count*100:.2f}%")
    print(f"  Half correct (genus only): {half_correct_species_top1}/{total_count}")
    print(f"  Half accuracy: {half_correct_species_top1/total_count*100:.2f}%")
    print(f"  Total with at least genus correct: {correct_species_top1 + half_correct_species_top1}/{total_count}")
    print(f"  Combined accuracy (full + half): {(correct_species_top1 + half_correct_species_top1)/total_count*100:.2f}%")

    print(f"\nInflorescence type Top-1 accuracy:")
    print(f"  Correct: {correct_inflorescence_top1}/{total_count}")
    print(f"  Accuracy: {correct_inflorescence_top1/total_count*100:.2f}%")

    # TOP-5 RESULTS
    print("\n" + "-" * 40)
    print("TOP-5 ACCURACY (Any of 5 predictions)")
    print("-" * 40)

    print(f"\nSpecies Top-5 accuracy:")
    print(f"  Full correct: {correct_species_top5}/{total_count}")
    print(f"  Full accuracy: {correct_species_top5/total_count*100:.2f}%")
    print(f"  Half correct (genus only): {half_correct_species_top5}/{total_count}")
    print(f"  Half accuracy: {half_correct_species_top5/total_count*100:.2f}%")
    print(f"  Total with at least genus correct: {correct_species_top5 + half_correct_species_top5}/{total_count}")
    print(f"  Combined accuracy (full + half): {(correct_species_top5 + half_correct_species_top5)/total_count*100:.2f}%")

    print(f"\nInflorescence type Top-5 accuracy:")
    print(f"  Correct: {correct_inflorescence_top5}/{total_count}")
    print(f"  Accuracy: {correct_inflorescence_top5/total_count*100:.2f}%")

    # COMPARATIVE SUMMARY
    print("\n" + "-" * 40)
    print("COMPARATIVE SUMMARY")
    print("-" * 40)

    print(f"\nSpecies (full match):")
    print(f"  Top-1: {correct_species_top1}/{total_count} ({correct_species_top1/total_count*100:.2f}%)")
    print(f"  Top-5: {correct_species_top5}/{total_count} ({correct_species_top5/total_count*100:.2f}%)")
    print(f"  Improvement: +{(correct_species_top5 - correct_species_top1)/total_count*100:.2f}% points")

    print(f"\nSpecies (genus only):")
    print(f"  Top-1: {half_correct_species_top1}/{total_count} ({half_correct_species_top1/total_count*100:.2f}%)")
    print(f"  Top-5: {half_correct_species_top5}/{total_count} ({half_correct_species_top5/total_count*100:.2f}%)")
    print(f"  Improvement: +{(half_correct_species_top5 - half_correct_species_top1)/total_count*100:.2f}% points")

    print(f"\nInflorescence type:")
    print(f"  Top-1: {correct_inflorescence_top1}/{total_count} ({correct_inflorescence_top1/total_count*100:.2f}%)")
    print(f"  Top-5: {correct_inflorescence_top5}/{total_count} ({correct_inflorescence_top5/total_count*100:.2f}%)")
    print(f"  Improvement: +{(correct_inflorescence_top5 - correct_inflorescence_top1)/total_count*100:.2f}% points")


# Run the tester with 50 images
llm_inference_tester(testsize=50)


Processing images (limit: 50)...

Image 1: train/1358101/4e0e883c4725a93fb91fedab55767fb57e2edf6c.jpg
True species: Sedum caeruleum | True inflorescence: Cyme
Getting 5 predictions...
  Prediction 1: Species: Sedum album | Inflorescence: Cyme
  Prediction 2: Species: Sedum anglicum | Inflorescence: Cyme
  Prediction 3: Species: Sedum hispanicum | Inflorescence: Cyme
  Prediction 4: Species: Sedum hirsutum | Inflorescence: Cyme
  Prediction 5: Species: Sedum album | Inflorescence: Cyme

Results:
  Species: Genus match only in top-5
  Inflorescence: Match in top-5
--------------------------------------------------------------------------------

Image 2: train/1405641/b739153b1f158e781663696512c872af3b769df6.jpg
True species: Asystasia gangetica | True inflorescence: Secund raceme
Getting 5 predictions...
  Prediction 1: Species: Asystasia gangetica | Inflorescence: Secund raceme
  Prediction 2: Species: Asystasia gangetica | Inflorescence: Secund raceme
  Prediction 3: Species: Asystasi